### Search Engine With Tools and Agents

In [22]:
## Arxiv -- Research
## Tools creation

from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [23]:
## Using inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki=wiki_query=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name


'wikipedia'

In [24]:
## Using inbuilt tool of arxiv
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv=arxiv_query=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name


'arxiv'

In [25]:
tools=[wiki,arxiv]

In [26]:
## Custom tools[RAG tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [27]:
import openai
import os

from dotenv import load_dotenv
load_dotenv()

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")


In [28]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectorstore=FAISS.from_documents(documents, OpenAIEmbeddings())
retriever=vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x13ca49db0>, search_kwargs={})

In [29]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever, "langsmith-search", "Search any information about Langsmith")
retriever_tool.name


'langsmith-search'

In [30]:
tools=[wiki,arxiv,retriever_tool]

In [31]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/a825765/Documents/me/NUS/Learn/Machine Learning/Langchain/venv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x12a35b5b0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS

In [32]:
## Run all the tools with Agents and LLM Models

## Tools , LLM --> Agent Executor
from langchain_groq import ChatGroq

groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")

In [33]:
## Prompt Template for ReAct Agent

from langchain import hub
# Use ReAct prompt which is compatible with most LLMs including Groq
react_prompt = hub.pull("hwchase17/react")
print("ReAct prompt template:")
print(react_prompt.template)

ReAct prompt template:
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [34]:
from langchain.agents import create_react_agent

# Use a different prompt template for ReAct agent
from langchain import hub
react_prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, react_prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith-search(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(tags=[\'FAISS\', \'OpenAIEmbeddings\'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x13ca49db0>, search_kwargs={}), document_prompt: \'BasePromptTemplate\' = PromptTe

In [35]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith-search(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(tags=[\'FAISS\', \'OpenAIEmbeddings\'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x13ca49db0>, search_kwarg

In [36]:
agent_executor.invoke({"input":"Tell me about LangSmith"})



> Entering new AgentExecutor chain...
Thought: LangSmith seems like a natural language processing (NLP) tool, so I should try to understand more about it.

Action: langsmith-search
Action Input: 'LangSmith'Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based o

{'input': 'Tell me about LangSmith',
 'output': 'LangSmith is a platform for building production-grade LLM applications, providing features for observability, evaluation, and prompt engineering to help developers closely monitor and evaluate their application, iterate on prompts, and optimize their LLM applications.'}

In [37]:
agent_executor.invoke({"input":"What is Machine Learning?"})



> Entering new AgentExecutor chain...
Thought: Machine Learning seems like a broad and complex topic that requires a comprehensive answer. I should start by searching for a general overview of Machine Learning.

Action: wikipedia
Action Input: "Machine Learning"Thought: Machine Learning seems like a broad and complex topic that requires a comprehensive answer. I should start by searching for a general overview of Machine Learning.

Action: wikipedia
Action Input: "Machine Learning"Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witPage: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witThought: The

{'input': 'What is Machine Learning?',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks with a high degree of accuracy.\n\nLet me explain the thought process and actions taken to arrive at the final answer:\n\n1. The initial question was "What is Machine Learning?" which seemed to require a comprehensive answer. I started by searching for a general overview of Machine Learning using Wikipedia.\n2. The result from Wikipedia provided a good starting point, so I decided to dig deeper and read the article to gain a better understanding of the topic.\n3. After reading the article, I felt that I had a good grasp of the basics of Machine Learning, so I decided to explore the history of Machine Learning to understand how it has evolved over time.\n4. However, the result from Wikipedia\'s "History of Machine Learning" p

In [38]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...
Thought: I need to find information about the paper with the arXiv ID 1706.03762.

Action: arxiv
Action Input: 1706.03762Thought: I need to find information about the paper with the arXiv ID 1706.03762.

Action: arxiv
Action Input: 1706.03762Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cPublished: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cHere's the answer:

Question: What's the paper 1706.03762 about?

Thought: I need to find information about the paper with the arXiv ID 1706.03762.

Action: arxiv
Action Input: 1706.03762Published: 2023-08-02
Title: At

{'input': "What's the paper 1706.03762 about?",
 'output': 'The paper 1706.03762 is titled "Attention Is All You Need" and it\'s about sequence transduction models, specifically that the dominant models are based on convolutional neural networks (CNNs) and recurrent neural networks (RNNs), but the authors propose a new model that only uses self-attention mechanisms, eliminating the need for these other components.'}